In [1]:
from functools import partial
from typing import Literal
from dataclasses import dataclass

import pandas as pd
import redis

In [2]:
r = redis.Redis(db=4)

keys = r.keys()

In [3]:
keys[0].decode("utf-8")

'جام\u200cجم گر طلبی مجلس ما را دریابکز گدایی در میکده سلطان شده\u200cایم'

In [4]:
r.hgetall(keys[0])

{b'm1': b'\xd8\xac\xd8\xa7\xd9\x85\xe2\x80\x8c\xd8\xac\xd9\x85 \xda\xaf\xd8\xb1 \xd8\xb7\xd9\x84\xd8\xa8\xdb\x8c \xd9\x85\xd8\xac\xd9\x84\xd8\xb3 \xd9\x85\xd8\xa7 \xd8\xb1\xd8\xa7 \xd8\xaf\xd8\xb1\xdb\x8c\xd8\xa7\xd8\xa8',
 b'm2': b'\xda\xa9\xd8\xb2 \xda\xaf\xd8\xaf\xd8\xa7\xdb\x8c\xdb\x8c \xd8\xaf\xd8\xb1 \xd9\x85\xdb\x8c\xda\xa9\xd8\xaf\xd9\x87 \xd8\xb3\xd9\x84\xd8\xb7\xd8\xa7\xd9\x86 \xd8\xb4\xd8\xaf\xd9\x87\xe2\x80\x8c\xd8\xa7\xdb\x8c\xd9\x85',
 b'poet': b'\xd9\x81\xd8\xb1\xd9\x88\xd8\xba\xdb\x8c \xd8\xa8\xd8\xb3\xd8\xb7\xd8\xa7\xd9\x85\xdb\x8c',
 b'url': b'https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh382/'}

### Here we write some structures and functions for handling Beyt objects

In [5]:
def load_beyt(data: dict[bytes, bytes]) -> dict[str, str]:
    return {k.decode("utf-8"): v.decode("utf-8") for k, v in data.items()}


@dataclass
class Beyt:
    m1: str
    m2: str
    poet: str
    url: str

    @property
    def key(self):
        return self.m1 + self.m2

In [6]:
Beyt(**load_beyt(r.hgetall(keys[0])))

Beyt(m1='جام\u200cجم گر طلبی مجلس ما را دریاب', m2='کز گدایی در میکده سلطان شده\u200cایم', poet='فروغی بسطامی', url='https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh382/')

In [7]:
db = []

for key in keys:
    db.append(Beyt(**load_beyt(r.hgetall(key))))

## Start Create DataFrame

In [8]:
df = pd.DataFrame(data=db)
df

,m1,m2,poet,url
0,جام‌جم گر طلبی مجلس ما را دریاب,کز گدایی در میکده سلطان شده‌ایم,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...
1,هزار عقل و ادب داشتم من ای خواجه,کنون که مست خرابم صلاح بی‌ادبیست,حافظ,https://ganjoor.net/hafez/ghazal/sh64/
2,چون سکندر تشنه‌لب بسیار دارم هر طرف,گر چه در ظلمت نهان چون آب حیوان مانده‌ام,صائب,https://ganjoor.net/saeb/divan-saeb/ghazal-sae...
3,قوت آمدنم نیست به نزد تو مگر,هم تو لطفی بکنی و به کرم باز آیی,اوحدی,https://ganjoor.net/ouhadi/divano/ghazalo/sh879/
4,به جهان ملک تویی بس نکشد کمان تو کس,بپرم چو تیر اگر تو به کمان ما درآیی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...
...,...,...,...,...
74170,بس کن که تلخ گردد دنیا بر اهل دنیا,گر بشنوند ناگه این گفت و گوی ما را,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh193/
74171,تا نیاید نگار ما در کار,کار ما چون نگار نتوان یافت,اوحدی,https://ganjoor.net/ouhadi/divano/ghazalo/sh144/
74172,ز دست عشق کی جسته‌ست تا جهد دل من,به قبض عشق بود قبضه قلاجوری,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh3...
74173,نصیحت گوش کن کاین در بسی به,از آن گوهر که در گنجینه داری,حافظ,https://ganjoor.net/hafez/ghazal/sh447/


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74175 entries, 0 to 74174
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   m1      74175 non-null  object
 1   m2      74175 non-null  object
 2   poet    74175 non-null  object
 3   url     74175 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [10]:
# Find Counts of Poems URL

df.url.value_counts()

## IF YOU COLLECTED ENORMOUS DATA OF A POEM URL THEN YOU CAUGHT ALL BEYTS(COUPLET)  :)))

url
https://ganjoor.net/hafez/saghiname/                  58
https://ganjoor.net/moulavi/shams/ghazalsh/sh1940/    46
https://ganjoor.net/hafez/ghaside/sh3/                43
https://ganjoor.net/hafez/ghaside/sh2/                40
https://ganjoor.net/hafez/ghaside/sh1/                40
                                                      ..
https://ganjoor.net/moulavi/shams/ghazalsh/sh773/      1
https://ganjoor.net/moulavi/shams/robaeesh/sh673/      1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh779/     1
https://ganjoor.net/salman/divanss/robaeess/sh106/     1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh514/     1
Name: count, Length: 14893, dtype: int64

In [11]:
# Get All Hafez Poems from db

df[df.poet == "حافظ"] # == df[df.poet.str.contains("حافظ")]

,m1,m2,poet,url
1,هزار عقل و ادب داشتم من ای خواجه,کنون که مست خرابم صلاح بی‌ادبیست,حافظ,https://ganjoor.net/hafez/ghazal/sh64/
50,راهم شراب لعل زد ای میر عاشقان,خون مرا به چاه زنخدان یار بخش,حافظ,https://ganjoor.net/hafez/ghazal/sh275/
65,رسد ز چرخ عطارد هزار تهنیتت,چو فکرتت صفت امر کن فکان گیرد,حافظ,https://ganjoor.net/hafez/ghaside/sh3/
74,بر تخت جم که تاجش معراج آسمان است,همت نگر که موری با آن حقارت آمد,حافظ,https://ganjoor.net/hafez/ghazal/sh171/
94,کوه صبرم نرم شد چون موم در دست غمت,تا در آب و آتش عشقت گدازانم چو شمع,حافظ,https://ganjoor.net/hafez/ghazal/sh294/
...,...,...,...,...
74138,که برد به نزد شاهان ز من گدا پیامی,که به کوی می فروشان دو هزار جم به جامی,حافظ,https://ganjoor.net/hafez/ghazal/sh468/
74153,چه ناله‌ها که رسید از دلم به خرمن ماه,چو یاد عارض آن ماه خرگهی آورد,حافظ,https://ganjoor.net/hafez/ghazal/sh147/
74158,چندان که گفتم غم با طبیبان,درمان نکردند مسکین غریبان,حافظ,https://ganjoor.net/hafez/ghazal/sh383/
74169,بجز ابروی تو محراب دل حافظ نیست,طاعت غیر تو در مذهب ما نتوان کرد,حافظ,https://ganjoor.net/hafez/ghazal/sh136/


#### cleaning m1, m1

In [12]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,m1,m2,poet,url
4665,...,زنده مانم چو در آمدز در معشوقی,محتشم کاشانی,https://ganjoor.net/mohtasham/divan-moh/ghazal-moh/sh574/
53667,***,ای فخر من سلطان من سلطان من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1785/


In [13]:
df.loc[53667, "m1"] = "ای فخر من سلطان من سلطان من"
df.loc[53667, "m2"] = "فرمان ده و خاقان من خاقان من"

df.drop(4665, inplace=True)

df.reset_index(inplace=True)
##  IRREVERSIBLE!

In [14]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,index,m1,m2,poet,url


In [15]:
df.loc[25377, "m2"] = "در فرقت آفتاب چون ماه مخسب"

df.loc[25377]

index                                               25378
m1                         ای دل دو سه شام تا سحرگاه مخسب
m2                             در فرقت آفتاب چون ماه مخسب
poet                                                مولوی
url      https://ganjoor.net/moulavi/shams/robaeesh/sh87/
Name: 25377, dtype: object

#### creating start, end word

In [16]:
# Start , Ends Cleaning


def clean_and_select(string: str, mode: Literal["start", "end"] = "start"):
    clean_dict = {
        "آ": "ا",
        "ء": "ا",
        "ة": "ت",
    }
    striped = string.strip(" \n\t.؟!،”:()«»")

    match mode:
        case "start":
            res = clean_dict.get(striped[0], striped[0])
        case "end":
            res = clean_dict.get(striped[-1], striped[-1])
        case _:
            raise ValueError(f'mode: Literal["start", "end"] (got {mode})')

    return res

In [17]:
df["start"] = df.m1.apply(partial(clean_and_select, mode="start"))
df["end"] = df.m2.apply(partial(clean_and_select, mode="end"))

df

,index,m1,m2,poet,url,start,end
0,0,جام‌جم گر طلبی مجلس ما را دریاب,کز گدایی در میکده سلطان شده‌ایم,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...,ج,م
1,1,هزار عقل و ادب داشتم من ای خواجه,کنون که مست خرابم صلاح بی‌ادبیست,حافظ,https://ganjoor.net/hafez/ghazal/sh64/,ه,ت
2,2,چون سکندر تشنه‌لب بسیار دارم هر طرف,گر چه در ظلمت نهان چون آب حیوان مانده‌ام,صائب,https://ganjoor.net/saeb/divan-saeb/ghazal-sae...,چ,م
3,3,قوت آمدنم نیست به نزد تو مگر,هم تو لطفی بکنی و به کرم باز آیی,اوحدی,https://ganjoor.net/ouhadi/divano/ghazalo/sh879/,ق,ی
4,4,به جهان ملک تویی بس نکشد کمان تو کس,بپرم چو تیر اگر تو به کمان ما درآیی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...,ب,ی
...,...,...,...,...,...,...,...
74169,74170,بس کن که تلخ گردد دنیا بر اهل دنیا,گر بشنوند ناگه این گفت و گوی ما را,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh193/,ب,ا
74170,74171,تا نیاید نگار ما در کار,کار ما چون نگار نتوان یافت,اوحدی,https://ganjoor.net/ouhadi/divano/ghazalo/sh144/,ت,ت
74171,74172,ز دست عشق کی جسته‌ست تا جهد دل من,به قبض عشق بود قبضه قلاجوری,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh3...,ز,ی
74172,74173,نصیحت گوش کن کاین در بسی به,از آن گوهر که در گنجینه داری,حافظ,https://ganjoor.net/hafez/ghazal/sh447/,ن,ی


In [18]:
# list of words that starts the m1
starts = df.start.value_counts()
starts, len(df.start.value_counts())

(start
 ا    10550
 ب     8686
 د     6420
 م     6000
 گ     4681
 چ     4547
 ه     3486
 ت     3319
 ز     2836
 ن     2727
 خ     2576
 س     2337
 ک     1974
 ش     1817
 ع     1617
 ر     1583
 ج     1392
 ی     1152
 و     1129
 پ     1073
 ح      822
 ص      774
 ف      699
 غ      526
 ق      518
 ل      471
 ط      314
 ذ       78
 ظ       34
 ض       21
 ث       11
 ژ        4
 Name: count, dtype: int64,
 32)

In [19]:
# list of words that ends the m2
ends = df.end.value_counts()
ends, len(df.end.value_counts())

(end
 د    15647
 ی    13464
 م    10181
 ت     9964
 ن     5820
 ا     4992
 ر     2521
 ش     2366
 ه     2274
 و     2251
 ل     1015
 ز      987
 ب      856
 س      385
 ق      283
 ک      268
 گ      139
 غ      131
 ف       97
 ح       94
 ع       94
 ج       80
 ظ       48
 خ       40
 ث       39
 چ       33
 ص       31
 ض       26
 ذ       24
 ط       24
 Name: count, dtype: int64,
 30)

In [20]:
end_scale = ends.sum() / ends
end_scale

end
د       4.740461
ی       5.509061
م       7.285532
ت       7.444199
ن      12.744674
ا      14.858574
ر      29.422451
ش      31.349958
ه      32.618294
و      32.951577
ل      73.077833
ز      75.150963
ب      86.651869
س     192.659740
ق     262.098940
ک     276.768657
گ     533.625899
غ     566.213740
ف     764.680412
ح     789.085106
ع     789.085106
ج     927.175000
ظ    1545.291667
خ    1854.350000
ث    1901.897436
چ    2247.696970
ص    2392.709677
ض    2852.846154
ذ    3090.583333
ط    3090.583333
Name: count, dtype: float64

In [21]:
# What word not exist in ends of m2?

set(df.start.unique()) - set(df.end.unique())

{'پ', 'ژ'}

In [22]:
#### variety of poetry

dfg = df.groupby("poet")

df_poet = dfg.agg("count").sort_values("m1")

df_poet

,index,m1,m2,url,start,end
poet,,,,,,
خیام,354,354,354,354,354,354
هاتف اصفهانی,559,559,559,559,559,559
رهی معیری,757,757,757,757,757,757
باباطاهر,760,760,760,760,760,760
عبید زاکانی,826,826,826,826,826,826
سیف فرغانی,1334,1334,1334,1334,1334,1334
ابوسعید ابوالخیر,1438,1438,1438,1438,1438,1438
شهریار,1507,1507,1507,1507,1507,1507
امیرخسرو دهلوی,1671,1671,1671,1671,1671,1671


In [23]:
df_poet.index

Index(['خیام', 'هاتف اصفهانی', 'رهی معیری', 'باباطاهر', 'عبید زاکانی',
       'سیف فرغانی', 'ابوسعید ابوالخیر', 'شهریار', 'امیرخسرو دهلوی', 'صائب',
       'عراقی', 'سلمان ساوجی', 'حافظ', 'اوحدی', 'خواجوی کرمانی',
       'محتشم کاشانی', 'فروغی بسطامی', 'سعدی', 'مولوی'],
      dtype='object', name='poet')

In [24]:
#### starts and ends in poetry of poet

sample_poet = df_poet.index[0]
print(sample_poet)
dfg.get_group(sample_poet).start.unique() , len(dfg.get_group(sample_poet).start.unique())

خیام


(array(['گ', 'چ', 'س', 'پ', 'د', 'خ', 'ر', 'ه', 'ف', 'ش', 'م', 'ک', 'ب',
        'ح', 'ا', 'ن', 'ی', 'و', 'ز', 'ت', 'ض', 'ص', 'ج', 'ق', 'ع', 'ل'],
       dtype=object),
 26)

In [25]:
# and ends
dfg.get_group(sample_poet).end.unique() , len(dfg.get_group(sample_poet).end.unique())

(array(['ن', 'و', 'د', 'ت', 'ی', 'م', 'ر', 'ه', 'ا', 'ش', 'ل', 'ب', 'س',
        'ز', 'گ', 'خ'], dtype=object),
 16)

In [26]:
# create counts of words in starts of each poets

starts_dict = {poet: len(dfg.get_group(poet).start.unique()) for poet in df_poet.index}
ends_dict = {poet: len(dfg.get_group(poet).end.unique()) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 16,
 'هاتف اصفهانی': 17,
 'رهی معیری': 15,
 'باباطاهر': 17,
 'عبید زاکانی': 13,
 'سیف فرغانی': 17,
 'ابوسعید ابوالخیر': 22,
 'شهریار': 16,
 'امیرخسرو دهلوی': 21,
 'صائب': 20,
 'عراقی': 18,
 'سلمان ساوجی': 17,
 'حافظ': 23,
 'اوحدی': 19,
 'خواجوی کرمانی': 23,
 'محتشم کاشانی': 29,
 'فروغی بسطامی': 15,
 'سعدی': 18,
 'مولوی': 25}

In [27]:
# create counts of words in starts of each poets

starts_dict = {poet: len(dfg.get_group(poet).start.unique()) for poet in df_poet.index}
ends_dict = {poet: len(dfg.get_group(poet).end.unique()) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 16,
 'هاتف اصفهانی': 17,
 'رهی معیری': 15,
 'باباطاهر': 17,
 'عبید زاکانی': 13,
 'سیف فرغانی': 17,
 'ابوسعید ابوالخیر': 22,
 'شهریار': 16,
 'امیرخسرو دهلوی': 21,
 'صائب': 20,
 'عراقی': 18,
 'سلمان ساوجی': 17,
 'حافظ': 23,
 'اوحدی': 19,
 'خواجوی کرمانی': 23,
 'محتشم کاشانی': 29,
 'فروغی بسطامی': 15,
 'سعدی': 18,
 'مولوی': 25}

In [28]:
df_count_alphabet_start = pd.DataFrame(
    data=starts_dict.values(),
    index=starts_dict.keys(),
    columns=["count_alphabet_start"],
)
df_count_alphabet_end = pd.DataFrame(
    data=ends_dict.values(),
    index=ends_dict.keys(),
    columns=["count_alphabet_end"],
)

df_count_alphabet = df_count_alphabet_start.merge(
    df_count_alphabet_end,
    left_index=True,
    right_index=True,
)

df_count = df_poet.merge(
    df_count_alphabet,
    left_index=True,
    right_index=True,
)[["start", "count_alphabet_start", "count_alphabet_end"]].rename(columns={"start": "poem_count"})

df_count

,poem_count,count_alphabet_start,count_alphabet_end
خیام,354,26,16
هاتف اصفهانی,559,26,17
رهی معیری,757,27,15
باباطاهر,760,27,17
عبید زاکانی,826,29,13
سیف فرغانی,1334,30,17
ابوسعید ابوالخیر,1438,30,22
شهریار,1507,29,16
امیرخسرو دهلوی,1671,29,21
صائب,2849,29,20


In [31]:
from functools import reduce

In [32]:
# create counts of words in starts of each poets

starts_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).start.unique())) for poet in df_poet.index}
ends_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).end.unique())) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 'ابتخدرزسشلمنهوگی',
 'هاتف اصفهانی': 'ابتجدرزسشقلمنهوگی',
 'رهی معیری': 'ابتحدرزشلمنهوکی',
 'باباطاهر': 'اتجدرزسشفلمنهوچگی',
 'عبید زاکانی': 'ابتدرزشلمنهگی',
 'سیف فرغانی': 'ابتجدرزشقلمنهوچکی',
 'ابوسعید ابوالخیر': 'ابتحدرزسشضطعقلمنهوچکگی',
 'شهریار': 'ابتجدرزسظلمنهوکی',
 'امیرخسرو دهلوی': 'ابتجحخدرزسشطعفقلمنهوی',
 'صائب': 'ابتحدرزسشعغقلمنهوکگی',
 'عراقی': 'ابتدرزسشغقلمنهوکگی',
 'سلمان ساوجی': 'ابتدرزسشعغقلمنهوی',
 'حافظ': 'ابتثجحخدرزسشعغفقلمنهوکی',
 'اوحدی': 'ابتحدرزسشقلمنهوچکگی',
 'خواجوی کرمانی': 'ابتحدرزسشصعغفقلمنهوچکگی',
 'محتشم کاشانی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوکگی',
 'فروغی بسطامی': 'ابتدرزشقلمنهوکی',
 'سعدی': 'ابتدذرزسشغقلمنهوگی',
 'مولوی': 'ابتثجحخدرزسشضعغفقلمنهوکگی'}

In [33]:
def remove_common_letters(dictionary):
    common_letters = set.intersection(*map(set, dictionary.values()))

    new_dictionary = {
        key: "".join(filter(lambda letter: letter not in common_letters, value))
        for key, value in dictionary.items()
    }

    return new_dictionary


remove_common_letters(starts_dict)
# remove_common_letters(ends_dict)

{'خیام': 'ض',
 'هاتف اصفهانی': 'غ',
 'رهی معیری': 'طغ',
 'باباطاهر': 'طغ',
 'عبید زاکانی': 'ذضطغ',
 'سیف فرغانی': 'ذطظغژ',
 'ابوسعید ابوالخیر': 'ذضطظغ',
 'شهریار': 'ثذطغ',
 'امیرخسرو دهلوی': 'ذطغژ',
 'صائب': 'ذضطغ',
 'عراقی': 'ذطظغ',
 'سلمان ساوجی': 'ذطظغ',
 'حافظ': 'ثذضطظغ',
 'اوحدی': 'ثذضطغ',
 'خواجوی کرمانی': 'ثذضطظغ',
 'محتشم کاشانی': 'ثذضطظغ',
 'فروغی بسطامی': 'ثذضطظغژ',
 'سعدی': 'ثذضطظغ',
 'مولوی': 'ذضطظغژ'}

In [34]:
poet_score_ends = {}

for poet, words in ends_dict.items():
    poet_score_end = 0
    for word in words:
        poet_score_end += end_scale[word]
    poet_score_ends[poet] = poet_score_end

df_final = df_count.merge(
    pd.DataFrame(
        poet_score_ends.values(),
        index=poet_score_ends.keys(),
        columns=["poet_score_ends"],
    ),
    right_index=True,
    left_index=True,
).sort_values(by=["poet_score_ends"])

In [35]:
df_final["score"] = (
    df_final["count_alphabet_start"]
    * df_final["count_alphabet_end"]
    * df_final["poet_score_ends"]
    / df_final["poem_count"]
)
df_final.sort_values("score")

,poem_count,count_alphabet_start,count_alphabet_end,poet_score_ends,score
فروغی بسطامی,5662,32,15,952.673042,80.763522
سلمان ساوجی,3343,29,17,2223.862972,327.958255
عراقی,3266,29,18,2245.172422,358.842622
سعدی,7282,31,18,5058.987098,387.656523
مولوی,22724,31,25,12124.291637,413.497889
عبید زاکانی,826,29,13,914.479767,417.383623
اوحدی,4805,30,19,4715.740757,559.411495
صائب,2849,29,20,3823.342634,778.356872
رهی معیری,757,27,15,1479.659208,791.627450
شهریار,1507,29,16,3324.350551,1023.555843
